In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

parameters = {
    'axes.labelsize':  14,
    'figure.titlesize': 16,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12,
    'axes.titlesize': 16,
    'legend.fontsize': 12,
    'legend.title_fontsize': 14
}
plt.rcParams.update(parameters)

from sklearn.metrics import mean_absolute_error

from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.preprocessing import PolynomialFeatures

Попробуйте создать искусственный датасет с лишними столбцами. Целевую метку, при правильной обработке данных, сформировать таким образом, чтобы без затруднений её смогла описать линейная модель. Данное задание не имеет какого-то “правильного” решения. Тем не менее, стоит представить те ситуации, которые могли бы быть в реальных данных.

### Создание семпла

Сделаем искусственный семпл описывающий размер заработной платы в зависимости от количества лет работы сотрудника, его уровня IQ пола, возраста.

In [9]:
# Создаём сэмпл
n_samples = 1000

# опыт работы в годах
experience_years = np.random.choice(20, n_samples)
# возраст сотрудника
age = np.random.choice(40, n_samples) + 18
# уровень IQ
IQ = np.random.choice(80, n_samples) + 70
# пол
sex = np.random.choice(1, n_samples)
# вес
hight = np.random.choice(70, n_samples) + 50

Искусственно сгенерируем велчину зарплаты. Она зависит от опыта и быстро растёт в первые годы рабты, замедляя свой рост в последующие. Эту закономерность отразим, взяв квадратный корень от опыта работы в годах. Также зарплата выше у сотрудников с более высоким IQ. От пола и роста у нас зарплата не зависит.

In [10]:
salary = 250*np.sqrt(experience_years) + IQ*500 + 20_000

data = pd.DataFrame({
    'experience': experience_years,
    'age': age,
    'IQ': IQ,
    'sex': sex,
    'hight': hight,
    'salary': salary
})
data.head(5)

,experience,age,IQ,sex,hight,salary
0,18,20,105,0,80,73560.660172
1,18,51,115,0,98,78560.660172
2,10,41,109,0,74,75290.569415
3,6,51,95,0,114,68112.372436
4,11,38,111,0,104,76329.156198


In [4]:
data.describe()

,experience,age,IQ,sex,hight,salary
count,1000.000000,1000.000000,1000.000000,1000.0,1000.000000,1000.000000
mean,9.203000,37.618000,109.142000,0.0,84.182000,75266.926712
std,5.866021,11.572467,23.268805,0.0,20.404245,11629.355893
min,0.000000,18.000000,70.000000,0.0,50.000000,55000.000000
25%,4.000000,27.000000,89.000000,0.0,66.000000,65347.454092
50%,9.000000,37.000000,109.000000,0.0,84.000000,75329.156198
75%,14.000000,48.000000,129.000000,0.0,102.000000,85433.613113
max,19.000000,57.000000,149.000000,0.0,119.000000,95589.724736


Построим линейную модель на основе всех данных

In [5]:
X = data[['experience', 'age', 'IQ', 'sex', 'hight']]
y = data['salary']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))

pred_values = reg.predict(X)
print('Bias: {}'.format(pred_values.mean() - y.mean()))

print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [4.95247290e+01 1.55384037e-02 4.99890497e+02 0.00000000e+00
 1.94534605e-02]
Bias: 0.0
Error: 60.84576349661296


Модель смогла понять, что ни возраст, ни пол, ни рост не имеют определяющего значения и приписала им очень малые веса.

In [6]:
X = data[['experience', 'IQ']]
y = data['salary']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))

pred_values = reg.predict(X)
print('Bias: {}'.format(pred_values.mean() - y.mean()))

print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [ 49.52405827 499.89257327]
Bias: 0.0
Error: 60.84114160897688


Если построить модель без лишних переменных, то ошибка окажется практически такой же, как и с ними.

In [30]:
poly = PolynomialFeatures(3)

In [32]:
X = data[['experience', 'age', 'IQ', 'sex', 'hight']]
X = poly.fit_transform(X)

y = data['salary']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))

pred_values = reg.predict(X)
print('Bias: {}'.format(pred_values.mean() - y.mean()))

print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [ 8.65390627e-09  1.42265349e+02 -5.77517204e+00  5.00652818e+02
  5.71372004e-08  2.89229576e+00 -8.51443434e+00  2.29284881e-01
 -1.37627449e-01  1.09597245e-08 -3.56236731e-02  1.07241015e-01
 -8.90610222e-03  6.52750465e-09  3.95105833e-02  1.26548910e-02
  2.32470004e-09 -2.32134013e-02 -3.52865751e-10 -5.31506561e-10
 -2.59195440e-02  2.15949199e-01 -3.33015083e-03  1.06704654e-03
  2.03627837e-10  3.10930221e-03 -6.48744965e-04 -7.86801267e-05
 -1.47952761e-11 -1.08962316e-03  3.15178417e-04 -9.37916411e-13
  3.63327690e-04  4.26325641e-14  0.00000000e+00 -2.01917511e-04
 -1.20110795e-03  3.31483160e-04  0.00000000e+00 -8.64540770e-05
 -1.15468731e-04  0.00000000e+00  1.08242592e-04  0.00000000e+00
  0.00000000e+00 -1.92154490e-04 -3.06298264e-05  0.00000000e+00
 -2.16073717e-05  0.00000000e+00  0.00000000e+00  1.08210221e-04
  0.00000000e+00  0.00000000e+00  0.00000000e+00  9.40125103e-05]
Bias: 1.4551915228366852e-11
Error: 19.56836433467336


Использование полиномиальных фич позволяет значительно снизить ошибку модели не внося заметного отклонения. Это происходит потому, что сумма степеней от опыта начинают аппроксимировать квадратный корень из опыта.

In [37]:
poly = PolynomialFeatures(3)

In [39]:
X = data[['experience', 'IQ']]
X['experience'] = np.sqrt(X['experience'])

y = data['salary']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))

pred_values = reg.predict(X)
print('Bias: {}'.format(pred_values.mean() - y.mean()))

print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [250. 500.]
Bias: 0.0
Error: 2.151500666514039e-11


<ipython-input-39-32abf1299554>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['experience'] = np.sqrt(X['experience'])


Замена опыта на квадратный корень из него позволяет уменьшить ошибку почти до нуля.